![Image](Images/Logo.png)



# How long will it take to dry my clothes?

### Introduction 

Climate change is an undeniable reality, but how can we (defeat it?)reduce our energetic consume? Firstly, we should overcome the excessive source waste by changing our daily habits. We propose an interactive website to take advantage of the best weather conditions in order to avoid using electricity for drying our clothes. It consists on an estimation of the time spent to dry the clothes we have washed and advice you when have probabilty of rain.  Using this website and bot we will help you to plan your laundry. 

### How it works? 

The weather data was acquired using the openweathermap’s API. Moreover, a physics model based on Dalton’s Law is used to calculate the time when your clothes are dry by introducing the type of clothes, their amount and the weather data. 

### Let's get started!


In [8]:
#LIBRARIES
from ipywidgets import interact, IntSlider
import numpy as np
import requests
import time
import ipywidgets

import nbinteract as nbi
import numpy as np

"""nbi:hide_in"""


appid1 = '3776d7611658054f9cdd2b4b06ef1bc3'
appid2 = ''
densityWater = 1000 #kg m⁻³
lavariable = 0.1

def evaporationRate(Tair, pressure, relativeHumidity,windSpeed, Rn):
  T = Tair
  RnMJm2day = Rn*10.08/117
  Delta = 5336/T**2*np.exp(21.07-5336/T)
  ea = np.exp(21.07-5336/T)
  D = (1-relativeHumidity)*ea
  lambd = 2.501 - 0.002361*(T-273)
  gamma = 0.0016286 * pressure / lambd
  return (Delta*RnMJm2day + gamma*(6.43*(1+0.5536*windSpeed)*D))/(lambd*(Delta+gamma))

#TIME
def dryingtime(E, massClothes, areaClothes):
  rhoE = 4.58e-5/3.957*E
  return massClothes/(4.58e-5/3.957*E*areaClothes)

class Weather():
  def __init__(self,**kwargs):
    for kwarg in kwargs:
      setattr(self,kwarg,kwargs[kwarg])
      
      if isinstance(kwargs[kwarg], list):
        if len(kwargs[kwarg]) == 1:
          kwargs[kwarg] = kwargs[kwarg][0]
      
      if isinstance(kwargs[kwarg], dict):
        for kkwarg in kwargs[kwarg]:
          setattr(self,kwarg+kkwarg,kwargs[kwarg][kkwarg])
  
  def __call__(self,**kwargs):
    for kwarg in kwargs:
      newarg = kwarg
      while hasattr(self,newarg):
        newarg = newarg + '_'
      setattr(self,newarg,kwargs[kwarg])
      
class Forecast():
    def __init__(self,*args):
      self.forecast = {}
      self.ll = []
      self.dd = []
      for arg in args:
        self.ll.append(Weather(**arg))
        self.dd.append(arg.get('dt_txt'))
      self.forecast = {key:val for key, val in zip(self.dd, self.ll)}
      
    def weatherid(self,mode=0):
      if mode==0:
        return [el.weatherid for el in self.ll]
      else:
        return {key: self.forecast.get(key).weatherid for key in self.forecast}
    
    def indfirstrain(self):
      wid = self.weatherid()
      for i in range(len(wid)):
        if wid[i] < 800:
          return i
    
    def warn(self):
      ind = self.indfirstrain()
      if not ind is None:
        date = self.dd[ind]
        desc = self.ll[ind].weatherdescription
        return date, desc
      else:
        return None, None
      
      
def getweath(city):
  """
  This function request the atmospheric live time data to the API 
  """
  resp = requests.get('http://api.openweathermap.org/data/2.5/weather?q={}&APPID={}'.format(city,appid1))
  if resp.status_code != 200:
    # This means something went wrong.
    print('Unavailable information')
    return None, None
  
  weath = Weather(**resp.json())
  #return weath

  
  lat = weath.coordlat
  lon = weath.coordlon
  resp2 = requests.get('http://api.openweathermap.org/data/2.5/uvi?lat={}&lon={}&APPID={}'.format(lat,lon,appid1))
  if resp.status_code != 200:
    # This means something went wrong.
    print('UV radiation not available')
    return weath, None
  
  weath(**resp2.json())
  
  resp3 = requests.get('http://api.openweathermap.org/data/2.5/forecast?q={}&APPID={}'.format(city,appid1))
  if resp3.status_code != 200:
    # This means something went wrong.
    print('Unavailable information')
    return weath, None
  
  forec = Forecast(*resp3.json().get('list'))
  
  return weath, forec
    
  
  
def demana_ciutat():
  city = str(input("Input city: "))
  return city

a1 = 0.5*.002 #m^3
m1 = 0.075 #kg
a2 = 1.44*.002 
m2 = 0.25 
slope = (m1/a1 + m2/a2)/2 #kg m^-3

clothlist = ['sockpair', 'underwear', 'tshirt', 'towel', 'sweater', 'trousers','bedsheet']
surfaces = dict(sockpair = 15*7*2*2*1e-4,\
                underwear = 20*40*2*1e-4,\
                tshirt = 50*40*2*1e-4,\
                towel = 40*150*1e-4,\
                sweater = 50*40*4*1e-4,\
                trousers = 120*20*4*1e-4,\
                bedsheet = 180*210*1e-4)

widths = dict(sockpair = 0.2*1e-2,\
                underwear = 0.2*1e-2,\
                tshirt = 0.2*1e-2,\
                towel = 0.6*1e-2,\
                sweater = 1*1e-2,\
                trousers = 0.5*1e-2,\
                bedsheet = 0.3*1e-2)
class Clothes:
  def __init__(self):
    self.contents = {}
  
  def __call__(self,req):
    return self.contents.get(req,None)
  
  def add(self,new,amount):
    self.contents[new] = Cloth(new,amount)
   
  def addmany(self,amounts):
    if len(amounts) != len(clothlist):
      print('Bad vector length')
      return None
    for i in range(len(amounts)):
      self.add(clothlist[i], amounts[i])
  
  def timeall(self,curweath):
    E = evaporationRate(curweath.maintemp, curweath.mainpressure/10, curweath.mainhumidity/100, curweath.windspeed, curweath.value*25*lavariable)
    return {el: dryingtime(E, self.__call__(el).waterw(), self.__call__(el).surface) for el in self.contents}
  
  def timemax(self,curweath):
    isthere = {el: int(bool(self.__call__(el).amount)) for el in self.contents}
    times = self.timeall(curweath)
    for el in self.contents:
      if isthere[el] == 0: times[el] = 0
    return max(list(times.values()))
    

class Cloth:
  def __init__(self,kind,amount):
    self.kind = kind
    self.amount = amount
    self.surface = surfaces.get(kind)
    self.width = widths.get(kind)
    self.volume = self.surface * self.width
  
  def waterw(self):
    return self.volume*slope
  
  
def main_program():
  """
  """
  ciutat = demana_ciutat()
  weath,forec = getweath(ciutat)
  
  
  #Tair, pressure, relativeHumidity, windSpeed, latitud, radiation = useful_data(weath)
  #evaporation_rate = evaporationRate(Tair, pressure, relativeHumidity,windSpeed,radiation)
  def showclothes(a,b,c,d,e,f,g):
    return a,b,c,d,e,f,g
  out = interact(showclothes,
        a=IntSlider(value=0, min=0, max=10, step=1, description='Sock pairs'),               
        b=IntSlider(value=0, min=0, max=10, step=1, description='Underwear'), 
        c=IntSlider(value=0, min=0, max=10, step=1, description='T-Shirts'),
        d=IntSlider(value=0, min=0, max=10, step=1, description='Towels'),
        e=IntSlider(value=0, min=0, max=10, step=1, description='Sweaters'), 
        f=IntSlider(value=0, min=0, max=10, step=1, description='Trousers'), 
        g=IntSlider(value=0, min=0, max=10, step=1, description='Bed Sheets'))
  
  from IPython.display import display
  button = widgets.Button(description="Continue")
  output = widgets.Output()

  display(button, output)

  def on_button_clicked(b):
    clothes = Clothes()
    clothes.addmany(out.widget.result)
    print('Drying time: {}'.format(time.strftime('%H:%M:%S', time.gmtime(clothes.timemax(weath)))))
    tup = forec.warn()
    print('Warning: on {}: there will be {}!'.format(tup[0], tup[1]))

  button.on_click(on_button_clicked)

  
  
main_program()

Input city: Barcelona


interactive(children=(IntSlider(value=0, description='Sock pairs', max=10), IntSlider(value=0, description='Un…

NameError: name 'widgets' is not defined

In [5]:
help(interact)

Help on _InteractFactory in module ipywidgets.widgets.interaction object:

class _InteractFactory(builtins.object)
 |  Factory for instances of :class:`interactive`.
 |  
 |  This class is needed to support options like::
 |  
 |      >>> @interact.options(manual=True)
 |      ... def greeting(text="World"):
 |      ...     print("Hello {}".format(text))
 |  
 |  Parameters
 |  ----------
 |  cls : class
 |      The subclass of :class:`interactive` to construct.
 |  options : dict
 |      A dict of options used to construct the interactive
 |      function. By default, this is returned by
 |      ``cls.default_options()``.
 |  kwargs : dict
 |      A dict of **kwargs to use for widgets.
 |  
 |  Methods defined here:
 |  
 |  __call__(self, _InteractFactory__interact_f=None, **kwargs)
 |      Make the given function interactive by adding and displaying
 |      the corresponding :class:`interactive` widget.
 |      
 |      Expects the first argument to be a function. Parameters to this